# Adding Confusion Species to Training Set


ANIA and AKEK were often false positives, this code is to get the 2021 XC snapshot recordings of ANIA and AKEK into the 5sec training set

In [1]:
import pandas as pd
from pathlib import Path
from glob import glob
from opensoundscape import CNN

Step 1: get the names of all audio files we will use

In [2]:
#gather ania XC mp3 names
ANIA_path = Path("/media/emu/datasets/annotated/xeno_canto_v2_1_2021/2021_12_04/magumma-parva/mp3s")
ANIA_recordings = glob(f"{ANIA_path}/*.mp3")

#gather akek XC mp3 names
AKEK_path = Path("/media/emu/datasets/annotated/xeno_canto_v2_1_2021/2021_12_04/loxops-caeruleirostris/mp3s")
AKEK_recordings = glob(f"{AKEK_path}/*.mp3")

#gather trainset mp3 names
KAAM_path_mac = Path("/home/dah238/Kauai-Amakihi/Audio/Macaulay Focal Recordings")
KAAM_path_XC = Path("/home/dah238/Kauai-Amakihi/Audio/Xeno-canto/mp3s")
KAAM_recordings = glob(f"{KAAM_path_mac}/*.wav") + glob(f"{KAAM_path_mac}/*.mp3") + glob(f"{KAAM_path_XC}/*")

fullaudionames = ANIA_recordings + AKEK_recordings + KAAM_recordings

step 2: create boxed annotations then make 5 second clip labels

In [3]:
from opensoundscape import BoxedAnnotations
# Set the current directory to where the dataset is downloaded
dataset_path = Path("/home/dah238/Kauai-Amakihi/Annotations")

# Make a list of all of the csvs utilized
selection_files = glob(f"{dataset_path}/*/*.csv")
selection_files

train = pd.concat((pd.read_csv("/home/dah238/Kauai-Amakihi/Annotations/macaulay/combined_output_macaulay_v3.csv"),
                   pd.read_csv("/home/dah238/Kauai-Amakihi/Annotations/xeno_canto/combined_output_xeno_canto.csv")))
train['annotation'] = train['annotation'].replace([' KAAM_song','KAAM_wsong'],'KAAM_song')
# train = train['annotation'].drop([])
train.annotation.value_counts()

annotation
KAAM_song        323
KAAM_call         80
?                 20
?_KAAM_song       16
?_KAAM_call        7
?KAAM_song         6
?KAAM_call         1
  ?_KAAM_song      1
Name: count, dtype: int64

In [4]:
train_boxed = BoxedAnnotations(df = train, audio_files = fullaudionames)
class_sub = ["KAAM_song","?KAAM_song","?_KAAM_song","  ?_KAAM_song"]

train_clips = train_boxed.clip_labels(
    clip_duration = 5, 
    clip_overlap = 2.5,
    min_label_overlap = 0.25,
    audio_files = fullaudionames,
    class_subset = class_sub
)


In [5]:
train_clips = train_clips[~(train_clips["  ?_KAAM_song"] | train_clips["?KAAM_song"] | train_clips["?_KAAM_song"])]
train_clips_pure = train_clips[["KAAM_song"]]

In [6]:
xc_files = (
    pd.read_csv(
        "/media/emu/datasets/annotated/xeno_canto_v2_1_2021/labels.csv", index_col=0
    )
    .sample(3000)
    .index.values
)
xc_labels = pd.DataFrame(
    {"file": xc_files, "start_time": 0, "end_time": 5, "KAAM_song": False}
).set_index(["file", "start_time", "end_time"])
train_w_xc = pd.concat([train_clips_pure, xc_labels])

In [7]:
#AKEK_tdl files
akek_tdl = pd.read_csv("/home/sml161/ecco14_hawaii/kaam/SL_KAAM_annots_for_akek-tdl.csv")
akek_tdl = akek_tdl.drop(columns=["comments","Unnamed: 0","AKEK_song","folder","point","notes"])
akek_tdl.to_csv("dow")
akek_tdl['annotation'] = akek_tdl['annotation'].replace('yes',True).replace('no',False)
akek_tdl = akek_tdl[akek_tdl['annotation'] != 'uncertain']
akek_tdl.rename(columns={'annotation':'KAAM_song'},inplace=True)
akek_tdl.set_index(["file", "start_time", "end_time"], inplace=True)
print(akek_tdl)
full_train = pd.concat((akek_tdl,train_w_xc))
full_train["KAAM_song"].value_counts()


                                                                       KAAM_song
file                                               start_time end_time          
/media/kiwi/datasets/finalized/kaua2024a/AKEK_g... 93         96            True
                                                   147        150           True
/media/kiwi/datasets/finalized/kaua2024a/AKEK_g... 78         81            True
/media/kiwi/datasets/finalized/kaua2024a/AKEK_g... 102        105           True
                                                   108        111           True
...                                                                          ...
/media/kiwi/datasets/finalized/kaua2024a/PUAI_O... 108        111           True
                                                   147        150           True
/media/kiwi/datasets/finalized/kaua2024a/PUAI_O... 15         18           False
/media/kiwi/datasets/finalized/kaua2024a/PUAI_O... 42         45            True
/media/kiwi/datasets/finaliz

KAAM_song
False    4016
True      836
Name: count, dtype: int64

In [20]:
valid_raw = pd.read_csv("/home/dah238/Kauai-Amakihi/Annotations/ania_tdl/SL_KAAM_annotations_from_ania_review_for_training (1).csv")
valid_raw.drop(columns=['clip_file',
 'clip_start_time',
 'clip_end_time',
 'comments',
 'ANIA_song',
 'comment',
 'clf_score',
 'deployment_id',
 'deployment_name',
 'project',
 'date',
 'recording_start_time',
 'datetime'],inplace=True)
valid_raw['KAAM_song'] = valid_raw["KAAM_song"].replace("no",False).replace("yes",True)
valid_raw = valid_raw[valid_raw["KAAM_song"].isin([True,False])]
valid_raw.set_index(["file", "start_time", "end_time"], inplace=True)



In [21]:
valid_raw["KAAM_song"].isna().sum()


np.int64(0)

This next chunk is to save the uncertains from akek_tdl to re annotate

In [9]:
#REANNOTATION INCOMPLETES
if False:
    uncertains = valid[(valid["annotation"] == "AKEK_KAAM") | (valid["annotation"] == "KAAM_ANIA") | (valid["annotation"] == "KAAM_song?")]
    uncertains.annotation.value_counts()
    uncertains["annotation"] = None
    uncertains.head()
    uncertains.rename(columns={"audio_file":"file"},inplace=True)


Back to chunks related to boxed annotations/clip labels for validation set

save as csv and inspect

In [ ]:
full_train.to_csv("/home/dah238/Kauai-Amakihi/Transfer_Learning/Experiment7_s/full_train.csv")

valid_raw.to_csv("/home/dah238/Kauai-Amakihi/Transfer_Learning/Experiment7_s/valid_ania.csv")

In [14]:
full_train.KAAM_song.value_counts()

KAAM_song
False    4016
True      836
Name: count, dtype: int64

In [16]:
valid_raw.KAAM_song.value_counts()

KAAM_song
False    4628
True       34
Name: count, dtype: int64